In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np

from trieste.acquisition.function.utils import *

%load_ext autoreload
%autoreload 2

2022-10-18 15:37:33.131768: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 15:37:33.890547: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9652 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-10-18 15:37:33.891125: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9652 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b3:00.0, compute capability: 7.5


In [2]:
def mc_mvn_cdf(
        mean: tf.Tensor,
        cov: tf.Tensor,
        x: tf.Tensor,
        num_samples: int,
    ):
    
    samples = tfd.MultivariateNormalFullCovariance(
        loc=mean,
        covariance_matrix=cov,
    ).sample(sample_shape=[num_samples])
    
    gt = tf.reduce_all(tf.math.less(samples, x[None, :, :]), axis=-1)
    
    return tf.cast(gt, dtype=tf.float32)

In [3]:
dtype = tf.float64
# B = 10
# Q = 3
S = 100
num_naive_samples = int(1e5)

mean = tf.convert_to_tensor(np.load("mean.npy"))
covariance = tf.convert_to_tensor(np.load("covariance.npy"))
threshold = tf.convert_to_tensor(np.load("threshold.npy"))

B, Q = mean.shape
samples = tf.math.sobol_sample(dim=Q, num_results=S, dtype=dtype)

print(mean.shape, covariance.shape, threshold.shape)

mvn_cdf = make_mvn_cdf(samples=samples)

(1, 5) (1, 5, 5) (1,)


In [4]:
qEI(
    mean=mean,
    covariance=covariance,
    threshold=threshold,
    mvn_cdf=mvn_cdf,
)

2022-10-18 15:37:34.309526: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x2380d440


2.300161412916041


<tf.Tensor: shape=(1,), dtype=float64, numpy=array([2.30016141])>